In [ ]:
from datetime import datetime, timedelta,date
import pandas as pd
import numpy as np
import jenkspy
from sklearn.model_selection import GridSearchCV
%matplotlib inline
from xgboost import plot_importance
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.cluster import KMeans
import warnings
warnings.filterwarnings("ignore")
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split
pyoff.init_notebook_mode()
import time
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
tx_train  = pd.read_csv('pjc_data.csv')
del tx_train['結帳日期']

In [ ]:
# list_cat = ['客戶自主下單','year','month','RecencyCluster','FrequencyCluster','RevenueCluster','OverallScore','Segment','cust_rank','分級']
# for i in list_cat:
#     tx_train[i] = tx_train[i].astype('category').cat.codes
#     tx_test[i] = tx_test[i].astype('category').cat.codes

In [ ]:
list_cat = ['客戶自主下單','year','month','RecencyCluster','FrequencyCluster','RevenueCluster','OverallScore','Segment','cust_rank','分級']
for i in list_cat:
    tx_train[i] = tx_train[i].apply(lambda x: "{}_{}".format(i, x))

In [ ]:
del tx_train["year"]

# onehot_encoding1=pd.get_dummies(tx_train["year"])
onehot_encoding2=pd.get_dummies(tx_train["month"])
onehot_encoding3=pd.get_dummies(tx_train["RecencyCluster"])
onehot_encoding4=pd.get_dummies(tx_train["FrequencyCluster"])
onehot_encoding5=pd.get_dummies(tx_train["OverallScore"])
onehot_encoding6=pd.get_dummies(tx_train["Segment"])
onehot_encoding7=pd.get_dummies(tx_train["cust_rank"])
onehot_encoding8=pd.get_dummies(tx_train["分級"])

dfa_firm_date = tx_train[['freq','max','總價','mean','med','Money','Recency','NextPurchaseDay']]

tx_train = pd.concat([dfa_firm_date,onehot_encoding2,onehot_encoding3,onehot_encoding4,onehot_encoding5,onehot_encoding6,onehot_encoding7,onehot_encoding8],axis=1)

In [ ]:
# x_train = tx_train.drop(['NextPurchaseDay'],axis=1)
# x_test = tx_test.drop(['NextPurchaseDay'],axis=1)

# y_train = tx_train['NextPurchaseDay']
# y_test = tx_test['NextPurchaseDay']

X = tx_train.drop(['NextPurchaseDay'],axis=1)
y = tx_train['NextPurchaseDay']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.25, random_state=1)
# x_train, y_train, x_valid, y_valid, x_test, y_test =  train_test_split(X, y, test_size=0.05, random_state=56)

In [ ]:
xgbr = xgb.XGBRegressor().fit(x_train, y_train)
score = xgbr.score(x_train, y_train)  
y_pred=xgbr.predict(x_test)
y_pred = np.round(y_pred)
score = xgbr.score(x_train, y_train)  
print("Training score: ", score)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
rmse=mse**0.5
print("RMSE: %.2f" % rmse)

In [ ]:
y_mean = [tx_train.loc[x,'mean'] for x in y_test.index]

In [ ]:
plt.figure(figsize=(20,4))
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, pd.Series(y_mean), label="predicted")
plt.legend()
plt.show()    

plt.figure(figsize=(20,4))
dtodraw = y_test.T.values[0].T - pd.Series(y_mean)
x_ax = range(len(y_test))
plt.plot(x_ax, dtodraw, label="diff")
plt.legend()
plt.show()
print(abs(dtodraw).sum())

In [ ]:
plt.figure(figsize=(20,4))
x_ax = range(len(y_test))
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, y_pred, label="predicted")
plt.legend()
plt.show()    

plt.figure(figsize=(20,4))
dtodraw = y_test.T.values[0].T - y_pred
x_ax = range(len(y_test))
plt.plot(x_ax, dtodraw, label="diff")
plt.legend()
plt.show()
print(abs(dtodraw).sum())

plot_importance(xgbr)
plt.show()

In [ ]:
cv_params = {'n_estimators': [300, 400, 500, 600, 700, 800]}
other_params = {'learning_rate': 0.1, 'n_estimators': 250, 'max_depth': 6, 'min_child_weight': 6, 'seed': 0,
'subsample': 0.8, 'colsample_bytree': 0.6, 'gamma': 0, 'reg_alpha': 1, 'reg_lambda': 0.1}
model = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4)
optimized_GBM.fit(x_train, y_train)
evalute_result = optimized_GBM.cv_results_
print('每輪迭代執行結果:{0}'.format(evalute_result))
print('引數的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

In [ ]:
xgbr = xgb.XGBRegressor(learning_rate = 0.1, n_estimators= 250, max_depth= 6, min_child_weight= 6, seed= 0,subsample= 0.8, colsample_bytree= 0.6, gamma= 0, reg_alpha= 1, reg_lambda= 0.1).fit(x_train, y_train)
score = xgbr.score(x_train, y_train)  
y_pred=xgbr.predict(x_test)
y_pred = np.round(y_pred)
score = xgbr.score(x_train, y_train)  
print("Training score: ", score)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print("MSE: %.2f" % mse)
rmse=mse**0.5
print("RMSE: %.2f" % rmse)